In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [2]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [3]:
ROOT_DIR = "/content/drive/MyDrive/Colab Notebooks/LG Aimers/data"
RANDOM_STATE = 110

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


## EDA

### 결측치 처리

40506 개의 중에 전부 NULL인 열 삭제

In [4]:
# 40506 개의 중에 전부 NULL인 열 삭제
train_data.dropna(axis=1, how='all', inplace=True)

In [5]:
# 삭제할 컬럼 목록
columns_to_delete = [
    'Wip Line_Dam', 'Process Desc._Dam', 'Insp Judge Code_Dam',
    'Wip Line_AutoClave', 'Process Desc._AutoClave', 'Equipment_AutoClave',
    'Insp Judge Code_AutoClave', '1st Pressure Judge Value_AutoClave',
    '2nd Pressure Judge Value_AutoClave', '3rd Pressure Judge Value_AutoClave',
    'Wip Line_Fill1', 'Process Desc._Fill1', 'Insp Judge Code_Fill1',
    'Wip Line_Fill2', 'Process Desc._Fill2', 'Insp Judge Code_Fill2',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill2',
    'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave',
    'Workorder_AutoClave', 'Workorder_Fill1', 'Workorder_Fill2'
]

# 해당 컬럼 삭제
train_data = train_data.drop(columns=columns_to_delete)

In [6]:
import pandas as pd

# 컬럼 목록
columns_to_modify = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'
]

# 'OK' 값을 None (NaN으로 처리)으로 변경
train_data[columns_to_modify] = train_data[columns_to_modify].applymap(lambda x: None if x == 'OK' else x)


<ipython-input-6-e561dbf4d0a7>:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_data[columns_to_modify] = train_data[columns_to_modify].applymap(lambda x: None if x == 'OK' else x)


In [7]:
# 모든 열을 비교하여 값이 동일한 열을 그룹으로 묶어주는 함수
def find_identical_columns(df):
    identical_columns = []
    checked_columns = set()

    for i, col1 in enumerate(df.columns):
        if col1 in checked_columns:
            continue

        group = [col1]
        for col2 in df.columns[i+1:]:
            if df[col1].equals(df[col2]):
                group.append(col2)
                checked_columns.add(col2)

        if len(group) > 1:
            identical_columns.append(group)

    return identical_columns

# 동일한 열을 그룹으로 출력
identical_column_groups = find_identical_columns(train_data)

# 중복 컬럼 그룹 예시 (실제 결과를 반영하여 생성)
duplicate_columns = identical_column_groups

# 함수: 중복되는 컬럼을 제거하고 하나의 컬럼만 남김
def drop_duplicate_columns(df, duplicate_columns):
    for group in duplicate_columns:
        # 그룹 내 첫 번째 컬럼만 남기고 나머지 컬럼 삭제
        columns_to_drop = group[1:]  # 첫 번째 컬럼을 제외한 나머지
        df = df.drop(columns=columns_to_drop)
    return df

# 중복 컬럼 제거
train_data = drop_duplicate_columns(train_data, duplicate_columns)


결측치 0으로 부분

In [8]:
# 적용할 컬럼 목록
columns_to_check = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
]

# 모든 컬럼에 대해 결측값을 0으로 대체
train_data[columns_to_check] = train_data[columns_to_check].fillna(0)

In [9]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# # 수치형으로 변환할 수 있는 컬럼 변환
# for col in columns_to_check:
#     try:
#         train_data[col] = pd.to_numeric(train_data[col])
#     except ValueError:
#         pass

In [10]:
# 수치형 컬럼 중 null이 있는 컬럼 확인
null_in_numeric = train_data.select_dtypes(include=['number']).isna().any()

# null 값이 있는 수치형 컬럼 출력
columns_with_null = null_in_numeric[null_in_numeric].index.tolist()

In [11]:
from sklearn.preprocessing import LabelEncoder

# 라벨 인코더 생성
label_encoder = LabelEncoder()

# target 변수 라벨 인코딩
train_data['target'] = label_encoder.fit_transform(train_data['target'])

### **원핫 인코딩 부분**

In [12]:
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

def parse_code(code):
    pattern = r"(\d)([A-Z0-9]{2})([A-Z0-9]+)-(\d+)"  # 숫자(1) 문자+숫자(2-3) 문자~숫자(4-8) - 숫자(9)
    match = re.match(pattern, code)
    if match:
        return match.groups()
    else:
        return None

# Workorder_Dam 컬럼을 파싱하여 새로운 컬럼으로 분할
parsed_columns = train_data['Workorder_Dam'].apply(parse_code)
train_data[['front_num', 'mid_str', 'tail_code', 'version']] = pd.DataFrame(parsed_columns.tolist(), index=train_data.index)

# 첫 번째 문자(숫자)와 두 번째 문자열에 대해 원핫 인코딩
onehot_encoder = OneHotEncoder(sparse_output=False)
front_num_encoded = onehot_encoder.fit_transform(train_data[['front_num']])
mid_str_encoded = onehot_encoder.fit_transform(train_data[['mid_str']])

# 숫자/문자 혼합에 대해 레이블 인코딩 수행
label_encoder = LabelEncoder()
tail_code_encoded = label_encoder.fit_transform(train_data['tail_code'])

# 인코딩된 결과를 데이터프레임에 추가
encoded_columns = pd.DataFrame(front_num_encoded, columns=[f'front_num_{i}' for i in range(front_num_encoded.shape[1])])
encoded_columns = encoded_columns.join(pd.DataFrame(mid_str_encoded, columns=[f'mid_str_{i}' for i in range(mid_str_encoded.shape[1])]))

# 컬럼 이름 변경하여 중복 회피
encoded_columns['tail_code_encoded'] = tail_code_encoded
encoded_columns['version_encoded'] = train_data['version'].astype(int)  # 버전은 수치형으로 변환

# 기존 컬럼 삭제 및 Workorder_Dam 컬럼 삭제
train_data.drop(columns=['front_num', 'mid_str', 'tail_code', 'version', 'Workorder_Dam'], inplace=True)

# 인코딩된 컬럼을 train_data에 병합
train_data = train_data.join(encoded_columns)


In [13]:
# 원핫인코딩 대상이 되는 오브젝트 타입 컬럼 선택
object_columns = train_data.select_dtypes(include=['object']).columns
print(object_columns)
# 원핫인코딩 수행
# 원핫인코딩 수행, 결과를 int로 바로 설정
train_data = pd.get_dummies(train_data, columns=object_columns)

Index(['Equipment_Dam', 'Model.Suffix_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam',
       'Chamber Temp. Judge Value_AutoClave', 'Equipment_Fill1',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
       'Equipment_Fill2',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'],
      dtype='object')


In [14]:
# 수치형 컬럼 선택 (float64와 int64 모두 포함)
numeric_columns = train_data.select_dtypes(include=['float64', 'int64']).columns

# 유니크한 값이 하나인 수치형 컬럼들을 찾고 삭제
columns_to_drop = [col for col in numeric_columns if train_data[col].nunique() == 1]

# 해당 컬럼 삭제
train_data.drop(columns=columns_to_drop, inplace=True)

# 삭제된 컬럼 리스트 출력
print(f"Deleted columns: {columns_to_drop}")

Deleted columns: ['Insp. Seq No._Dam', 'CURE STANDBY POSITION X Collect Result_Dam', 'CURE STANDBY POSITION Z Collect Result_Dam', 'CURE STANDBY POSITION Θ Collect Result_Dam', 'CURE END POSITION Θ Collect Result_Fill2', 'CURE STANDBY POSITION X Collect Result_Fill2', 'CURE START POSITION Θ Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill2']


In [15]:
import pandas as pd

def find_identical_numeric_columns(df):
    # 수치형 컬럼만 선택 (float64와 int64 모두 포함)
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

    identical_columns = []
    checked_columns = set()

    for i, col1 in enumerate(numeric_columns):
        if col1 in checked_columns:
            continue

        group = [col1]
        for col2 in numeric_columns[i+1:]:
            if df[col1].equals(df[col2]):
                group.append(col2)
                checked_columns.add(col2)

        if len(group) > 1:
            identical_columns.append(group)

    return identical_columns

# 동일한 수치형 컬럼을 그룹으로 묶어서 출력
identical_column_groups = find_identical_numeric_columns(train_data)

# 결과 확인
identical_column_groups


[]

수치형 컬럼 중복제거.

In [16]:
!pip install imbalanced-learn
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from pycaret.classification import *

# 데이터 분리
X = train_data.drop('target', axis=1)
y = train_data['target']

# 원핫 인코딩된 컬럼과 수치형 컬럼 분리
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
non_numeric_columns = X.select_dtypes(exclude=['float64', 'int64']).columns

# 수치형 컬럼에 대해서만 스케일링 적용
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X[numeric_columns])
X_numeric_scaled = pd.DataFrame(X_numeric_scaled, columns=numeric_columns)

# 스케일링 되지 않은 컬럼들 (원핫 인코딩된 컬럼 등) 병합
X_scaled = pd.concat([X_numeric_scaled, X[non_numeric_columns].reset_index(drop=True)], axis=1)

# 데이터셋을 80% 학습용, 20% 테스트용으로 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# SMOTE를 사용하여 학습 데이터에 대해 오버샘플링 수행
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 학습 데이터와 테스트 데이터를 하나의 데이터프레임으로 결합
train_data_resampled = pd.concat([X_train_resampled, y_train_resampled], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

# PyCaret 환경 설정
clf1 = setup(data=train_data_resampled, target='target', session_id=42, train_size=0.8)

# 여러 모델을 비교하여 최적의 모델 선택
best_model = compare_models()

# 선택된 최적 모델 출력
print(best_model)

# 선택된 모델로 테스트 데이터 예측
predictions = predict_model(best_model, data=test_data)

# 성능 평가
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

y_test = test_data['target']
y_pred = predictions['Label']

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


,Description,Value
0,Session id,42
1,Target,target
2,Target type,Binary
3,Original data shape,"(61048, 175)"
4,Transformed data shape,"(61048, 175)"
5,Transformed train set shape,"(48838, 175)"
6,Transformed test set shape,"(12210, 175)"
7,Numeric features,141
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9848,0.9981,0.9828,0.9868,0.9848,0.9697,0.9697,20.2400
rf,Random Forest Classifier,0.9744,0.9954,0.9936,0.9569,0.9749,0.9488,0.9495,18.7400
xgboost,Extreme Gradient Boosting,0.9682,0.9833,0.9973,0.9424,0.9691,0.9364,0.9380,5.0490
lightgbm,Light Gradient Boosting Machine,0.9678,0.9819,0.9994,0.9400,0.9688,0.9356,0.9375,14.8320
dt,Decision Tree Classifier,0.9346,0.9346,0.9265,0.9418,0.9341,0.8692,0.8694,2.7070
gbc,Gradient Boosting Classifier,0.9196,0.9662,0.9966,0.8636,0.9253,0.8392,0.8493,52.5840
knn,K Neighbors Classifier,0.8874,0.9576,0.8104,0.9580,0.8780,0.7748,0.7842,8.9800
ada,Ada Boost Classifier,0.8380,0.9082,0.8905,0.8059,0.8460,0.6759,0.6799,10.0990
lda,Linear Discriminant Analysis,0.6954,0.7770,0.8044,0.6605,0.7254,0.3909,0.4005,1.8160
lr,Logistic Regression,0.6901,0.7679,0.7871,0.6592,0.7175,0.3801,0.3875,19.1580


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     monotonic_cst=None, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=42, verbose=0,
                     warm_start=False)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.9309,0.6437,0.9814,0.9471,0.9640,0.1270,0.1407


KeyError: 'Label'